I've tried reproducing Andrew Ng's version (and couldn't get it to work). <br>
Then I followed the geeks for geeks tutorial, and was able to get it to work. <br>
Now I'll try to create an independent version.

In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
import pprint
%matplotlib inline

In [ ]:
# import VGG 19 model and keras Model API
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

<h3> Load images </h3>

<h5> Load content image </h5>

In [ ]:
img_size = 400
content_path = "louvre.jpg"
# original image is (300, 400, 3)

content_image = np.array(Image.open(content_path).resize((img_size, img_size)))   # Now shape is 400, 400, 3
content_image = np.reshape(content_image, ((1,) + content_image.shape ))                # Now (1, 400, 400, 3)                
content_image = tf.constant(content_image)    # Convert to tensor

print(content_image.shape)
imshow(content_image[0])
plt.show()

<h5> Load style image </h5>

In [ ]:
# original image once again (300, 400, 3)

style_path = "van_gogh.jpg"

style_image = np.array(Image.open(style_path).resize((img_size, img_size)))
style_image = tf.constant(np.reshape(style_image, ((1,) + style_image.shape)))

print(style_image.shape)
imshow(style_image[0])
plt.show()

<h5> Preprocess images </h5>

In [ ]:
# Convert the values in content_image to float32 types between 0 and 1. Then save as tf Variable.
preprocessed_content = tf.Variable (tf.image.convert_image_dtype(content_image, tf.float32))        # (1, img_size, img_size, 3)

# Convert the values in style_image to float32 types between 0 and 1. Then save as tf Variable.
preprocessed_style = tf.Variable (tf.image.convert_image_dtype(style_image, tf.float32))            # (1, img_size, img_size, 3)

<h3> Load model </h3>

In [ ]:
vgg_model = VGG19(
    include_top=False,
    weights='imagenet'
)
# Freeze weights
vgg_model.trainable = False
 
for layer in vgg_model.layers:
    print(layer.name)

In [ ]:
# For the content layer, we will use 'block5_conv4' 
# For the style layers, we will use ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

In [ ]:
vgg_model.input, vgg_model.get_layer('block1_conv1').output

In [ ]:
content_layer = 'block5_conv4'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

content_model = Model(inputs = vgg_model.input, outputs = vgg_model.get_layer(content_layer).output)
style_models = [Model(inputs = vgg_model.input, outputs = vgg_model.get_layer(style_layer).output) for style_layer in style_layers]



<h3> Cost functions </h3>

<h5> 1) Content Cost </h5>

In [ ]:
def content_cost (content, generated):
    """
    content, generated = 4D tensors
    """
    n_H, n_W, n_C = content[0].shape                    # content = (1,400,400,3)
    a_C = content_model(content)[0]
    a_G = content_model (generated)[0]
    content_cost = (1/(4*n_H*n_W*n_C))* tf.reduce_sum(tf.square(tf.subtract(a_C, a_G)))
    return content_cost

<h5> 2) Style Cost </h5>

In [ ]:
def gram_matrix (A):
    gram = tf.linalg.matmul(A, tf.transpose(A))
    return gram

def style_cost(style_image, generated_image, style_weights = [0.2, 0.2, 0.2, 0.2, 0.2]):

    """
    inputs
    style_image = preprocessed_image, 4D tensor (1, n_H, n_W, n_C), with values from 0 to 1

    return
    total cost
    """

    J_style_total = 0

    for i, style_model in enumerate(style_models):
        a_S = style_model(style_image)[0]       # a_S = (n_H, n_W, n_C)
        a_G = style_model(generated_image)[0]   # a_G = (n_H, n_W, n_C)

        n_H, n_W, n_C = a_S.shape
        
        # Reshape the images from (n_H * n_W, n_C) to have them of shape (n_C, n_H * n_W)
        a_S = tf.reshape(a_S, shape = [n_H * n_W, n_C])
        a_S = tf.transpose(a_S, perm = [1,0])   # a_S = (n_C, n_H * n_W)

        a_G = tf.reshape(a_G, shape = [n_H * n_W, n_C])
        a_G = tf.transpose(a_G, perm = [1,0])   # a_G = (n_C, n_H * n_W)

        # Compute gram_matrices for both images S and G 
        GS = gram_matrix(a_S)
        GG = gram_matrix(a_G)

        # Compute style cost for current layer
        J_style_layer = (1/(4*n_C*n_C*(n_H*n_W)**2)) * tf.reduce_sum(tf.multiply(tf.subtract(GS, GG), tf.subtract(GS, GG)))

        # Update total

        J_style_total += style_weights[i]*J_style_layer

    return J_style_total




<h3> Necessary Post-processing Functions </h3>

In [ ]:
def clip_0_1(image):
    """
    Truncate all the pixels in the tensor to be between 0 and 1
    During optimisation, we want to make sure the updated pixel values for the image stay within the range 0 to 1.

    Returns:
    Tensor
    """
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def tensor_to_image(tensor):
    """
    Scales the tensor values (input values are between 0 and 1) back to range of 0 to 255.
    Also convert the values into integers.
    If tensor is 4D, then take only last 3 values.
    Convert this to PIL image.
    
    Arguments:
    tensor -- Tensor
    
    Returns:
    Image: A PIL image
    """
    tensor = tensor * 255                       # Convert values to range 0–255
    tensor = np.array(tensor, dtype=np.uint8)   # Convert into integer type
    if np.ndim(tensor) > 3:                     # If more than 3 dimensions, take only last 3 dimensions.
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

<h3> Optimise </h3>

In [ ]:
type(preprocessed_content)

In [ ]:
content = tf.identity(preprocessed_content)
style = tf.identity(preprocessed_style)
generated = tf.Variable(tf.identity(content))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)  

with tf.GradientTape() as tape:
    J_content = content_cost(content, generated)
    J_style = style_cost(style, generated)
    J_total = 10 * J_content + 1000 * J_style

grads = tape.gradient(J_total, generated)           # J_total is y, generated is x; 
print (content[0,0,0], generated[0,0,0])
opt.apply_gradients([(grads, generated)])
print(content[0,0,0], generated[0,0,0])


In [ ]:
content [0,0,0], preprocessed_content[0,0,0]
content = content + 1
content [0,0,0], preprocessed_content[0,0,0]

In [ ]:
generated_images = []
def training_loop(preprocessed_content, preprocessed_style, iterations=500, a=10, b=1000, learning_rate = 0.1):
    # Notice that the weight for the style error is way bigger. Compensates for not adding noise to initial generated image.
    
    # load content and style images from their respective path
    content = tf.identity(preprocessed_content)         # use tf.identity to create deep copy
    style = tf.identity(preprocessed_style)
    generated = tf.Variable(tf.identity(content))       # use tf.Variable since "generated" will be updated
 
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)  
 
    best_cost = 1000000000000000             # set to a huge number (original had "Inf")
    best_image = None
    for i in range(iterations):
        
        with tf.GradientTape() as tape:
            J_content = content_cost(content, generated)
            J_style = style_cost(style, generated)
            J_total = a * J_content + b * J_style
 
        grads = tape.gradient(J_total, generated)           # J_total is y, generated is x; 
        opt.apply_gradients([(grads, generated)])           # optimise x so as to minimise y.
        generated.assign(clip_0_1(generated))

        generated_images.append(generated)

        if J_total < best_cost:
            best_cost = J_total
            best_image = generated
        
        if i%1000==0:
            image = tensor_to_image(best_image)
            print(f'Iteration: {i}')
            print(f'Total Loss: {J_total}')
            imshow(image)
            plt.show()

    
    best_image = tensor_to_image(best_image)                # Convert format of image (currently a tensor) ot image
 
    return best_image

In [ ]:
best_image1 = training_loop(preprocessed_content, preprocessed_style, iterations=100, a=10, b=1000, learning_rate = 0.07)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (12,6), dpi = 300)
ax[0].imshow(content_image[0])
ax[0].set_title('Content image')
ax[1].imshow(style_image[0])
ax[1].set_title('Style image')
ax[2].imshow(best_image1)
ax[2].set_title('Generated image v1')
ax[2].set_xlabel('iterations: 100\n a: 10\n b: 1000\n learning_rate: 0.07')
plt.tight_layout()
plt.savefig('generated_image_v1.jpg')
plt.show()

In [ ]:
best_image2 = training_loop(preprocessed_content, preprocessed_style, iterations=1000, a=10, b=100, learning_rate = 0.05)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (12,6), dpi = 300)
ax[0].imshow(content_image[0])
ax[0].set_title('Content image')
ax[1].imshow(style_image[0])
ax[1].set_title('Style image')
ax[2].imshow(best_image2)
ax[2].set_title('Generated image v2')
ax[2].set_xlabel('iterations: 1000\n a: 10\n b: 100\n learning_rate: 0.05')
plt.tight_layout()
plt.savefig('generated_image_v2.jpg')
plt.show()

In [ ]:
best_image3 = training_loop(preprocessed_content, preprocessed_style, iterations=10000, a=10, b=100, learning_rate = 0.001)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (12,6), dpi = 300)
ax[0].imshow(content_image[0])
ax[0].set_title('Content image')
ax[1].imshow(style_image[0])
ax[1].set_title('Style image')
ax[2].imshow(best_image3)
ax[2].set_title('Generated image v3')
ax[2].set_xlabel('iterations: 10,000\n a: 10\n b: 100\n learning_rate: 0.001')
plt.tight_layout()
plt.savefig('generated_image_v3.jpg')
plt.show()

In [ ]:
best_image4 = training_loop(preprocessed_content, preprocessed_style, iterations=15000, a=10, b=300, learning_rate = 0.001)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (12,6), dpi = 300)
ax[0].imshow(content_image[0])
ax[0].set_title('Content image')
ax[1].imshow(style_image[0])
ax[1].set_title('Style image')
ax[2].imshow(best_image4)
ax[2].set_title('Generated image v4')
ax[2].set_xlabel('iterations: 15,000\n a: 10\n b: 300\n learning_rate: 0.001')
plt.tight_layout()
plt.savefig('generated_image_v4.jpg')
plt.show()

In [ ]:
best_image5 = training_loop(preprocessed_content, preprocessed_style, iterations=25000, a=10, b=1000, learning_rate = 0.001)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (12,6), dpi = 300)
ax[0].imshow(content_image[0])
ax[0].set_title('Content image')
ax[1].imshow(style_image[0])
ax[1].set_title('Style image')
ax[2].imshow(best_image5)
ax[2].set_title('Generated image v5')
ax[2].set_xlabel('iterations: 25,000\n a: 10\n b: 1000\n learning_rate: 0.001')
plt.tight_layout()
plt.savefig('generated_image_v5.jpg')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,5, figsize = (20,5), dpi = 150)
ax[0].imshow(best_image1)
ax[1].imshow(best_image2)
ax[2].imshow(best_image3)
ax[3].imshow(best_image4)
ax[4].imshow(best_image5)
plt.show()

In [ ]:
best_image1.save("just_generated_image_v1.jpg")
best_image2.save("just_generated_image_v2.jpg")
best_image3.save("just_generated_image_v3.jpg")
best_image4.save("just_generated_image_v4.jpg")
best_image5.save("just_generated_image_v5.jpg")